## Placing automated orders in AlgoTading & fetching live streaming stock market data

This notebook tells how to establish connection between python & zerodha, how to fetch real time streaming data and most importantly how to place orders to buy/sell stocks based on time & price

1. Generating Request & Access tokens

2. Fetching Last Traded Price (ltp)

3. Fetching Interval High Price & Interval Low Price

4. Fetching live streaming tick level data using Websocket

5. Placing Orders based on Price

6. Placing Orders based on Time

## 1.  Generating Request & Access tokens.

In [1]:
from selenium import webdriver
from time import sleep
from kiteconnect import KiteConnect
from kiteconnect import KiteTicker
from pyotp import TOTP

import json
import time
import datetime
import pandas as pd

In [2]:
with open('credentials_zerodha.json','r') as file:
    login = json.load(file)

username       = login['user_id']
password       = login['password']
totp           = login['totp']
webdriver_path = login['webdriver_path'] #path of your chromedriver
api_key        = login['api_key']
api_secret     = login['api_secret']

#### Open Zerodha login page using selenium webdriver 

In [8]:
# launch chrome and open zerodha website

kite = KiteConnect(api_key)

service = webdriver.chrome.service.Service('./chromedriver.exe')
service.start() 

options = webdriver.ChromeOptions()
options = options.to_capabilities()

driver  = webdriver.Remote(service.service_url, options)
driver.get(kite.login_url())
driver.maximize_window()

C:\Users\ganesh\AppData\Local\Temp/ipykernel_636/2650630266.py:15: DeprecationWarning: desired_capabilities has been deprecated, please pass in an Options object with options kwarg
  driver  = webdriver.Remote(service.service_url, options)


#### Login using credentials

In [9]:
#input username
user = driver.find_element_by_xpath("//input[@type = 'text']")
user.send_keys(username)

#input password
pwd = driver.find_element_by_xpath("//input[@type = 'password']")
pwd.send_keys(password)

#click on login
driver.find_element_by_xpath("//button[@type='submit']").click()

sleep(1)

#input totp
ztotp      = driver.find_element_by_xpath("//input[@type = 'text']")
totp_token = TOTP(totp)
token      = totp_token.now()
ztotp.send_keys(token)

#click on continue
driver.find_element_by_xpath("//button[@type = 'submit']").click()

C:\Users\ganesh\AppData\Local\Temp/ipykernel_636/3302311798.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  user = driver.find_element_by_xpath("//input[@type = 'text']")
C:\Users\ganesh\AppData\Local\Temp/ipykernel_636/3302311798.py:6: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  pwd = driver.find_element_by_xpath("//input[@type = 'password']")
C:\Users\ganesh\AppData\Local\Temp/ipykernel_636/3302311798.py:10: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath("//button[@type='submit']").click()
C:\Users\ganesh\AppData\Local\Temp/ipykernel_636/3302311798.py:15: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  ztotp      = driver.find_element_by_xpath("//input[@type = 'text']")
C:\

#### Get Request & Access token

In [11]:
sleep(2)

#request token
request_token = driver.current_url.split('request_token=')[1].split('&')[0]

#access token
token = kite.generate_session(request_token, api_secret=api_secret)
access_token = token['access_token']

#connect python with Kite API
kite.set_access_token(access_token)

#closing chrome
driver.quit()

## 2. Fetching the Last Traded Price (ltp)

LTP is Last Traded Price i.e. most recent price of any stock.

#### a. LTP through Kite Connect API

In [13]:
kite.set_access_token(access_token)

In [14]:
ticker = "NSE:INFY"
kite.ltp(ticker)[ticker]["last_price"]

1472

In [16]:
ticker = "NFO:BANKNIFTY22JUNFUT"
kite.ltp(ticker)[ticker]["last_price"]

33281.65

#### b. LTP through 'pynse' library

In [19]:
from pynse import *

nse = Nse()            #create an nse object

nse.get_quote("INFY")["lastPrice"]

1472.5

In [18]:
quote = nse.get_quote("BANKNIFTY",Segment.FUT)
quote["lastPrice"]

33285.7

## 3. Fetching Interval High Price & Interval Low Price

Interval High & Low are the highest & lowest price of any stock in a particular interval. Interval high/low plays an important role in algorithmic trading. Many strategies are based on this high / low factor.

#### a. Interval High/Low through Kite

In [26]:
ticker = "NSE:INFY"

interval_high = kite.quote(ticker)[ticker]['ohlc']['high']
print(f'interval high for {ticker} =',interval_high)

interval_low  = kite.quote(ticker)[ticker]['ohlc']['low']
print(f'interval low for {ticker} =',interval_low)

interval high for NSE:INFY = 1474.3
interval low for NSE:INFY = 1457


#### b. Interval High/Low through pynse library

In [28]:
from pynse import *

quote = nse.get_quote('INFY')

interval_high = quote['high']
print('interval high =', interval_high)

interval_low = quote['low']
print('interval low =',interval_low)

interval high = 1474.3
interval low = 1457


## 4. Tick level data through Websocket API

WebSocket API is the most efficient (speed, latency, resource consumption, and bandwidth) way to receive real time quotes for instruments across all exchanges during live market hours. A quote consists of fields such as open, high, low, close, ltp, etc.

In [31]:
tokens = [738561, 5633] #token numbers of instruments

def on_ticks(ws, ticks):
    print(ticks)

def on_connect(ws, response):
    # Subscribe to a list of instrument_tokens 
    ws.subscribe(tokens)
    ws.set_mode(ws.MODE_FULL, [tokens[0]])

NOTE :

1. The token numbers that are being passed into the on_connect() function must be in a list. Even if it is subscribing to only a single instrument.

2. In on_connect() function, mode can be set to ws.MODE_LTP which will only fetch ltp or ws.MODE_QUOTE which will fetch open, high, low, close, volume. Here we set ws.MODE_FULL which will fetch every detail of instrument.

In [34]:
# Assign the callbacks.
kws = KiteTicker(token['api_key'], access_token)

kws.on_ticks   = on_ticks
kws.on_connect = on_connect
kws.connect()

[{'tradable': True, 'mode': 'full', 'instrument_token': 738561, 'last_price': 2561.45, 'last_traded_quantity': 9, 'average_traded_price': 2533.71, 'volume_traded': 3856865, 'total_buy_quantity': 308327, 'total_sell_quantity': 300908, 'ohlc': {'open': 2507.2, 'high': 2563.95, 'low': 2501.0, 'close': 2527.7}, 'change': 1.3352059184238636, 'last_trade_time': datetime.datetime(2022, 6, 29, 11, 50, 54), 'oi': 0, 'oi_day_high': 0, 'oi_day_low': 0, 'exchange_timestamp': datetime.datetime(2022, 6, 29, 11, 50, 54), 'depth': {'buy': [{'quantity': 19, 'price': 2560.9, 'orders': 1}, {'quantity': 1, 'price': 2560.85, 'orders': 1}, {'quantity': 60, 'price': 2560.8, 'orders': 2}, {'quantity': 89, 'price': 2560.75, 'orders': 3}, {'quantity': 59, 'price': 2560.7, 'orders': 1}], 'sell': [{'quantity': 115, 'price': 2561.9, 'orders': 2}, {'quantity': 2, 'price': 2561.95, 'orders': 2}, {'quantity': 996, 'price': 2562.0, 'orders': 6}, {'quantity': 4, 'price': 2562.15, 'orders': 1}, {'quantity': 119, 'price'

Connection error: 1006 - connection was closed uncleanly (SSL error: unexpected eof while reading (in ))
Connection closed: 1006 - connection was closed uncleanly (SSL error: unexpected eof while reading (in ))


## 5. Placing Price based Orders through Kite

- Let’s say after thorough investigation of historical data, it has been been observed that after reaching a certain price, the price increases sharply. In this case, it is profitable if one buys after reaching trigger price and then sell it.

- If price of Glenmark rises after it reaches 450 points, one could place buy order for intraday trading as shown below…

In [ ]:
ticker_symbol  = "NSE:GLENMARK"
trading_symbol = 'GLENMARK'
quantity       = 1

for i in range(10000):
    
    ltp = kite.ltp(ticker_symbol)[ticker_symbol]["last_price"]
   
    if ltp >= 450:
        kite.place_order(tradingsymbol = trading_symbol,
                             exchange  = "NSE",
                      transaction_type = "BUY",
                              quantity = quantity,
                            order_type = "MARKET",
                               variety = "regular",
                               product = "MIS")
        break
        
    time.sleep(5)

- In above function, a long for loop is running where its fetching the ltp of Glenmark every 5 seconds. As soon as the if condition is satisfied, i.e. price breaches 450 points, the if statement is executed. Kite’s place_order() function is used to place the BUY order. After place_order() is executed, the loop breaks.

- Similarly one can place ‘SELL’ order i.e. short if it is observed that price drops after reaching certain price. “BUY” can be replace with “SELL” in transaction_type in place_order() function to place sell order.

## 6. Placing Time based Orders through Kite

- Let’s say it is observed that price of the stock falls after 1 p.m. Here too we can modify our code such that order is placed based on time.

- If Glenmark price falls after 1 p.m., one can short sell it and place sell order. After some time, it can be bought again to book profit for an intraday trade.

In [ ]:
ticker_symbol  = "NSE:GLENMARK"
trading_symbol = "GLENMARK"
quantity       = 1

#The time at which we want to place order (24 hr format) 
entry_hour   = 13
entry_minute = 00

for i in range(1000000):
    time = datetime.datetime.now()
    
    if (time.hour == entry_hour) and (time.minute == entry_minute):
        kite.place_order(tradingsymbol = trading_symbol,
                             exchange  = "NSE",
                      transaction_type = "SELL",
                              quantity = quantity,
                            order_type = "MARKET",
                               variety = "regular",
                               product = "MIS")        
        
        print('order executed at', f'{entry_hour}:{entry_minute}')
        break
    time.sleep(5)

Above function runs a loop where it check for trigger time every 5 seconds. As soon as it triggers, if condition satisfies and place_order() function is executed and sell order is placed. Buying it again after some time will result in profit if strategy works.